# __QLSTM__
> Point Forecasting
- point forecasting은 실제 값과 다를 가능성이 매우 크다. 사실상 항상 다르다.
> Probabilistic Forecasting
- 실제 값이 높은 확률로 어느 범위에 속할지 예측해보자.
- 보통은 예측으로 끝이 아니고 예측 값을 가지고 무언가 분석 또는 스케줄링 등을 수행할 것이다.
- 구간 값을 얻음으로써 어떤 분석에서의 Best case와 Worst Case를 얻을 수 있다.
- 따라서 미래에 대해 더욱 유연한 대처가 가능하다.

## __Start__
- Multi-Step 예측


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
seed = 511
torch.manual_seed(seed)
np.random.seed(seed)

# Hyperparameter
BATSIZE = 32

## __GPU__

In [3]:
print("============================================================================================")
# set device to cpu or cuda
device = torch.device('cpu')
if(torch.cuda.is_available()):
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
print("============================================================================================")

Device set to : NVIDIA GeForce RTX 2080 Ti


## __Data Loading__

In [4]:
df = pd.read_csv('D:/Github/DeepLearning/load.csv')
df = df.iloc[:, 2:]

df.head(3)

,date,load,year,month,hour,day,lowtmep,hightemp
0,2016/11/25 0:00,1.357909,0.0,0.909091,0.000000,0.666667,0.302326,0.243902
1,2016/11/25 1:00,1.309840,0.0,0.909091,0.043478,0.666667,0.302326,0.243902
2,2016/11/25 2:00,1.326598,0.0,0.909091,0.086957,0.666667,0.302326,0.243902


## __Preprocessing__

In [5]:
X = df.iloc[:, 1:].values
Y = df.iloc[:, 1].values.reshape(-1, 1)


showme = 24*60   # 실제 예측이 이뤄지는 구간
inputlen = 24*7 # 인풋으로 사용할 구간

xtrain = X[:-(inputlen + showme), :]
ytrain = Y[:-(inputlen + showme), :]

xtest = X[-(inputlen + showme):, :] # 실제 예측 구간은 showme 구간인데, showme 앞부분을 넣어야 showme 부분에 대해 예측할 수 있음.
ytest = Y[-(inputlen + showme):, :] # ytest에서는 사실 inputlen 필요 없는데, Loader에 넣을 때 data 길이 같아야 해서 넣었음.
#                                     예측 후 시각화 할 때는 inputlen 부분 제거하고 그려야 함.

# 아래와 같은 sklearn 사용은 단순 regression에서의 사용이 적절함.
#xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=seed, shuffle=False)

print(f'xtrain.shape:  {xtrain.shape}')
print(f'xtest.shape:  {xtest.shape}')
print()
print(f'ytrain.shape:  {ytrain.shape}')
print(f'ytest.shape:  {ytest.shape}') # 24*7 + 24*14 = 504 (실제로는 24*14만 사용.)

xtrain.shape:  (24672, 7)
xtest.shape:  (1608, 7)

ytrain.shape:  (24672, 1)
ytest.shape:  (1608, 1)


In [6]:
xscaler = MinMaxScaler()
yscaler = MinMaxScaler()

# fit: xtrain을 기준으로 변환하겠다.
# transform: ㄹㅇ로 변환 그 자체만 수행.
# fit_transform(kane): kane을 기준으로, 그리고 찐으로 변환 수행.
xscaler.fit(xtrain)
scaled_xtrain = xscaler.transform(xtrain)
scaled_xtest = xscaler.transform(xtest)

yscaler.fit(ytrain)
scaled_ytrain = yscaler.transform(ytrain)
scaled_ytest = yscaler.transform(ytest)

In [7]:
def seq_data(x, y, xseq_len, yseq_len):  # e.g. N=100, in=5, out=3

    xseqlist = []
    yseqlist = []
    it = (len(x) - xseq_len) // yseq_len # 몫

    for i in range(it):
        xseqlist.append(x[i*yseq_len: i*yseq_len + xseq_len])
        yseqlist.append(y[xseq_len + i*yseq_len: xseq_len + i*yseq_len + yseq_len])
        # e.g.
        # x[0*3: 0*3 + 5]        =x[0:5],     x[1*3: 1*3 + 5]        =x[3:8] 
        # y[5 + 0*3: 5 + 0*3 + 3]=y[5:8],     y[5 + 1*3: 5 + 1*3 + 3]=y[8:11]
        #
        # x[92:97]
        # y[97:100]

    xxx = torch.tensor(np.array(xseqlist), dtype=torch.float32).to(device)
    yyy = torch.tensor(np.array(yseqlist), dtype=torch.float32).to(device)

    return xxx, yyy

In [8]:
xseq_len = 24*7
yseq_len = 24*1

xtrain_seq, ytrain_seq = seq_data(x=scaled_xtrain, y=scaled_ytrain, xseq_len=xseq_len, yseq_len=yseq_len)
xtest_seq, ytest_seq = seq_data(x=scaled_xtest, y=scaled_ytest, xseq_len=xseq_len, yseq_len=yseq_len)

print(f'xtrain_seq:   {xtrain_seq.shape}')
print(f'ytrain_seq:   {ytrain_seq.shape}')
print()
print(f'xtest_seq:    {xtest_seq.shape}')
print(f'ytest_seq:    {ytest_seq.shape}') # 매일 24시간에 대해 14일 동안 보는 중!

xtrain_seq:   torch.Size([1021, 168, 7])
ytrain_seq:   torch.Size([1021, 24, 1])

xtest_seq:    torch.Size([60, 168, 7])
ytest_seq:    torch.Size([60, 24, 1])


## __Loader__

In [9]:
# sequential data라 모든 epoch에서 같은 패턴만 보게 됨.,
# 모델이 시계열의 순서 그 자체를 학습하게 될 우려가 있음. ----> shuffle=True
trainset = TensorDataset(xtrain_seq, ytrain_seq)
trainloader = DataLoader(trainset, batch_size=BATSIZE, shuffle=True)

# Validation과 Test는 말 그대로 평가의 의미가 있으므로 
# 같은 패턴을 유지해야 비교할 수 있음. ----> shuffle=False
testset = TensorDataset(xtest_seq, ytest_seq)
testloader = DataLoader(testset, batch_size=BATSIZE, shuffle=False)

In [10]:
for i in trainloader:
    print(i[0].shape)
    print(i[1].shape)
    break
print()
#=============================
for j in testloader:
    print(j[0].shape) # 이전 24*7시간 사용.
    print(j[1].shape) # 매일 24시간에 대해 14일 동안 예측 수행.

torch.Size([32, 168, 7])
torch.Size([32, 24, 1])

torch.Size([32, 168, 7])
torch.Size([32, 24, 1])
torch.Size([28, 168, 7])
torch.Size([28, 24, 1])


## shuffle 예시
- input과 label pair만 항상 붙어다님.
- 하나의 batch 구성은 계속 바뀜.
- shuffle=True로 하면 각 size 32일 때, 32개가 무작위로 뽑혀오는 거지 time series의 sequence 자체가 무너지는 건 아님.

In [11]:
x = torch.arange(6).view(-1, 1).float()  # feature: 0부터 4까지의 숫자
y = x * 200
dataset = TensorDataset(x, y)

dataloader_shuffle = DataLoader(dataset, batch_size=2, shuffle=True)
dataloader_no_shuffle = DataLoader(dataset, batch_size=2, shuffle=False)

for i in dataloader_shuffle:
    print(i)
#-------------------------------
print()
for i in dataloader_no_shuffle:
    print(i)

[tensor([[2.],
        [1.]]), tensor([[400.],
        [200.]])]
[tensor([[5.],
        [0.]]), tensor([[1000.],
        [   0.]])]
[tensor([[4.],
        [3.]]), tensor([[800.],
        [600.]])]

[tensor([[0.],
        [1.]]), tensor([[  0.],
        [200.]])]
[tensor([[2.],
        [3.]]), tensor([[400.],
        [600.]])]
[tensor([[4.],
        [5.]]), tensor([[ 800.],
        [1000.]])]


## __Model__
> self.lstm.flatten_parameters()
- lstm의 파라미터들이 GPU 상에서 메모리에 분산 배치되는 것을 방지.(한 곳에 모아줌.)


In [12]:
# out, (hn, cn) = self.lstm
# out: containing the output features (h_t) from the last layer of the LSTM, for each t.
# hn:  containing the final hidden state for each element in the sequence.
# cn:  containing the final cell state for each element in the sequence.
# 출처 - 파이토치 공식 문서

class LSTMModel(nn.Module):
    def __init__(self, input_len, hidden_len, seq_len, num_layers, output_len, num_quantiles):
        super(LSTMModel, self).__init__()
        self.input_len = input_len
        self.hidden_len = hidden_len
        self.seq_len = seq_len
        self.num_layers = num_layers
        self.output_len = output_len
        self.num_quantiles = num_quantiles
        self.lstm = nn.LSTM(self.input_len, self.hidden_len, self.num_layers, batch_first=True, dropout=0.3) # model.eval() 사용 시 Dropout은 자동 해제됨.
        self.linear = nn.Linear(self.hidden_len*self.seq_len, self.output_len*self.num_quantiles)
        self.relu = nn.ReLU()

    def forward(self, x):
        self.lstm.flatten_parameters()
        # x:  (64, 24*7, 6) ----> feature 6개가 하나로 7일치를 묶음. 64개 샘플.
        h0 = torch.zeros((self.num_layers, x.shape[0], self.hidden_len)).to(device) 
        c0 = torch.zeros((self.num_layers, x.shape[0], self.hidden_len)).to(device)
        # h0: (2, 64, 8) ----> hidden 8개를 64개 샘플에 대해, two layer로.
        # c0: (2, 64, 8) ----> hidden 8개를 64개 샘플에 대해, two layer로.
        out, (hn, cn) = self.lstm(x, (h0,c0)) 
        # out: (64, 24*7, 8) ----> 각각들이 hidden을 8개씩 출력했고 각 시간 단위마다 출력함.
        #                                                입력 시퀀스의 각 타임스텝에 대응되는 출력을 내보냄.
        out = out.reshape(out.shape[0], -1)
        # out: (64, 24*7*8)   
        out = self.linear(out) 
        # out: (64, 24*9)    
        out = self.relu(out)
        # out: (64, 24*9)    ----> 24시간씩 9개의 Quantile을 가짐. 
        out = out.reshape(out.shape[0], self.output_len, self.num_quantiles)
        # out: (64, 24, 9)

        return out

#=========================================================================================================    
#=========================================================================================================
def pinball_loss(preds, target, quantiles):
    losslist = []

    # preds: (64, 24, 9)  ----> 24시간씩 9개 Quantile 예측.
    # target: (64, 24, 1) ----> 실제 값은 하나.

    for i, q in enumerate(quantiles):
        error = target - preds[:, :, i].unsqueeze(2) # 하나 꺼내오면 (64, 24)라서 unsqueeze(2) 해줬음.
        losslist.append( torch.max((q-1)*error,  q*error).unsqueeze(2) )
    loss = torch.mean(torch.cat(losslist, dim=2))

    return loss


#````````````````````````````````````````````````````````````````````````````````````````````````````````
# 하나의 target에만 의존하여 모든 Quantile이 계산되는데, 
# 서로 독립적으로 처리되므로 Quantile Crossing 문제 발생.
        
#````````````````````````````````````````````````````````````````````````````````````````````````````````

# 왜 코드에서 (q-1)을 쓰는가?
    
# 0.9 quantile일 때 실제보다 underestimation한 경우, penalty 강하게 줘야.
# ====> error는 음수,   max(-0.1*양수,  0.9*양수)  =  0.9*양수(큰 penalty)
    
# 0.9 quantile일 때 실제보다 overestimation한 경우, penalty 적게 줘야.
# ====> error는 양수,   max(-0.1*음수,  0.9*음수) =  -0.1*음수(작은 penalty)

# torch.max( , ) --> 한 줄로 Loss는 언제나 양수가 나온다.
# 또한 penalty 정도를 알아서 잘 나눠준다.
    

In [13]:
model = LSTMModel(input_len=xtrain_seq.shape[2], hidden_len=8, seq_len=xtrain_seq.shape[1],\
                num_layers=2, output_len=ytrain_seq.shape[1], num_quantiles=9).to(device)

criterion = nn.L1Loss().to(device) # MAE
optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-5)

## __Training__

In [14]:
loss_graph = []
n = len(trainloader)
quantiles = [kane/10  for kane in range(1, 10)]
#``````````````````````````````````````````````````````````````

for epoch in range(1, 700+1):
    running_loss = 0.0
    
    for x_true, y_true in trainloader:
        # x_true: (64, 24*7, 6)
        # y_true: (64, 24, 1)

        y_pred = model(x_true) 
        # y_pred: (64, 24, 1)

        loss = pinball_loss(y_pred, y_true, quantiles)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    loss_graph.append(running_loss/n)
    if epoch % 100 == 0:
        print('[epoch: %d] loss: %.4f' %(epoch, running_loss/n))

[epoch: 100] loss: 0.0155
[epoch: 200] loss: 0.0150
[epoch: 300] loss: 0.0149
[epoch: 400] loss: 0.0147
[epoch: 500] loss: 0.0145
[epoch: 600] loss: 0.0143
[epoch: 700] loss: 0.0142


## __Training Curve__

In [15]:
plt.figure(figsize=(12,6))
plt.plot(loss_graph)
plt.show() 

## __Test__

In [16]:
test_loss_list = []
epoch_loss = 0

# 안에 있는 각 list가 각 quantile들의 결과를 가지고 있음.
predlist = [ [] for _ in range(len(quantiles))]

model.eval()
with torch.no_grad():
    for x, ytrue in testloader:

        ypred = model(x)
        # y_pred: (14, 24, 9)


        for i in range(len(quantiles)):
            predlist[i] += ypred[:, :, i].flatten().cpu().tolist() 

        loss = pinball_loss(ypred, ytrue, quantiles)

        epoch_loss += loss.item()
    
    test_loss_list.append(epoch_loss/n)


#=======================================================================================================
print(f'Loss:  {np.round(test_loss_list, 7)[0]}')

truelist = ytest.flatten().tolist()[inputlen:]

for i in range(len(quantiles)):
    predlist[i] = yscaler.inverse_transform(np.array(predlist[i]).reshape(-1, 1)).ravel().tolist()

Loss:  0.0012842


## Save Quantiles

In [58]:
'''
kane = pd.DataFrame()
for i in range(len(quantiles)):
    
    kane[f'{i+1}'] = np.array(predlist[i])

display(kane.head())
'''

"\nkane = pd.DataFrame()\nfor i in range(len(quantiles)):\n    \n    kane[f'{i+1}'] = np.array(predlist[i])\n\ndisplay(kane.head())\n"

In [57]:
#kane.to_csv('Quantiles_Load.csv', index=False)

## __Plot__

In [43]:
num = len(quantiles)
s = 24*39
e = s + 24*7
#````````````````````````````````````````````````````````````````````````````````````````````````
fig, ax = plt.subplots(1,1,   figsize=(32, 10))


for i in range(round(num/2)):
    plt.fill_between(range(len(predlist[0]))[s:e],  
                    predlist[i][s:e], predlist[num-(i+1)][s:e], 
                    color='cornflowerblue', alpha=0.15, edgecolor='blue') # Quantiles


ax.plot(range(len(predlist[0]))[s:e], predlist[4][s:e], color='blue',   linestyle='dashed', label='Median') # Median

ax.plot(range(len(predlist[0]))[s:e], truelist[s:e], color='k',  linewidth=2.0, label='True') # True

#====================================================================================================

plt.legend(fontsize=18, frameon=True)
ax.set_xticks(range(s, e, 24))
ax.set_xticklabels([f'Day{i+1}' for i in range( int((e-s)/24) )], fontsize=16)

ax.set_title('Quantile Forecasting of Load', fontsize=24)
ax.set_xlabel('Days', fontsize=18) 
ax.set_ylabel('Load [kWh]', fontsize=18) 

#plt.savefig('Load.png', dpi=300)

plt.show()

In [18]:
sum_weekday = [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 62, 62, 62, 62, 62, 25, 25, 25,]
sum_weekend = [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 38, 38, 38, 38, 38, 25, 25, 25,]

win_weekday = [23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 59, 59, 59, 59, 59, 23, 23, 23,]
win_weekend = [23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 59, 59, 59, 59, 59, 23, 23, 23,]

## __Results__

## __End__